# Select Your IPTS 

In [1]:
from __code import system
system.System.select_working_dir()

** Using Debugging Mode! **


HBox(children=(Label(value='Select Working Folder', layout=Layout(width='20%')), Select(options=('df', 'ob', 'sample'), value='df')))

# Description 

This notebook performs the normalization of a set of images (tiff or fits) by OBs. 

The steps are:
 - select your **sample images**
 - select your **ob images**
 - select your **df images** (optional)
 - all the data will be loaded
 - preview of the sample and ob
 - selection of a **background** region in the sample data
 - normalization is performed
 - export of the normalized data

# Python Import

In [2]:
from __code.__all import custom_style
custom_style.style()

In [3]:
# from __code.normalization import NormalizationHandler

In [4]:
%matplotlib notebook

In [5]:
# from ipywidgets import widgets, Layout
# from IPython.core.display import display
# import ipywe.fileselector


# Select Images 

In [6]:
from __code.normalization_new_ui import *

In [7]:
files = Files()
sample_panel = SampleSelectionPanel(working_dir=system.System.get_working_dir())
sample_panel.init_ui(files=files)
wizard = WizardPanel(sample_panel=sample_panel)

HTML(value='Please wait...')

Label(value='Selection of All Input Files', layout=Layout(border='1px lighgray solide', height='35px', padding='8px', width='300px'))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Sample files ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Label(value='         ', layout=Layout(width='70%')), Button(button_style='info', description='Next Step>>', disabled=True, layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/git/testing_data/df', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' DC0000.tif     |     Sat Jul 22 10:10:32 2017', ' DC0001.tif     |     Sat Jul 22 10:10:32 2017', ' DC0002.tif     |     Sat Jul 22 10:10:32 2017'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

...

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/git/testing_data', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .git          |     Directory', ' README.md     |     Tue Nov 21 12:10:06 2017', ' df            |     Directory', ' ob            |     Directory', ' sample        |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Dark Field files ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Button(button_style='info', description='<< Previous Step', layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to previous step'), Label(value='         ', layout=Layout(width='70%'))))

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/git/testing_data', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .git          |     Directory', ' README.md     |     Tue Nov 21 12:10:06 2017', ' df            |     Directory', ' ob            |     Directory', ' sample        |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

# Loding data 

In [ ]:
o_norm_handler.load_data()

# Preview Data

In [ ]:
o_norm_handler.plot_images(data_type='sample')

In [ ]:
o_norm_handler.plot_images(data_type='ob')

# Select Background Region 

In [ ]:
o_norm_handler.with_or_without_roi()

In [ ]:
o_norm_handler.select_sample_roi()

# Normalization

In [ ]:
o_norm_handler.run_normalization()

# Export 

In [ ]:
o_norm_handler.select_export_folder()

In [ ]:
o_norm_handler.export()